In [2]:
import pandas as pd
import joblib

# Cargar el modelo y los encoders
model = joblib.load('car_price_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Cargar el dataset para sacar luego los modelos correspondientes a la marca que se introduzca
df = pd.read_csv('car_price_dataset.csv')

# Sacar la lista de variables categóricas que existan en el dataset
marcas_validas = list(label_encoders['Brand'].classes_) # .classes_ (Atributo de LabelEncoder, array con todas los tipos de marcas únicas en este caso)
combustibles_validos = list(label_encoders['Fuel_Type'].classes_)
transmisiones_validas = list(label_encoders['Transmission'].classes_)

# Función para solicitar entrada validada
def solicitar_input(mensaje, tipo, valid_values=None): # valid_values: Lista de valores válidos para entradas tipo str. Si no se pone se ajusta por defecto a None.
    while True: # Se repite hasta que el usuario ingrese un valor válido.
        valor = input(mensaje).strip() # .strip(): Elimina espacios en blanco al inicio y final.
        if tipo == int:
            if not valor.isdigit(): # isdigit() verifica si la entrada contiene solo dígitos. Es más eficiente que un try except.
                print("Error: Debes ingresar un número entero.")
                continue # Valor inválido, en python 'continue' hace que el bucle avance a la siguiente iteración
            return int(valor)
        
        elif tipo == float:
            try:
                return float(valor)
            except ValueError:
                print("Error: Debes ingresar un número válido con decimales (ej: 1.8).")
                continue

        elif tipo == str:
            if valid_values and valor not in valid_values: # (None se interpreta como false.)
                print(f"Error: El valor '{valor}' no es válido. Opciones disponibles: {', '.join(valid_values)}") # join conecta cada valor seprado por ,
                continue
            return valor

# Pedir la marca del coche
marca_seleccionada = solicitar_input(f"Introduce la marca del coche ({', '.join(marcas_validas)}): ", str, marcas_validas)

# Filtrar modelos que pertenecen a la marca seleccionada
modelos_validos = df[df['Brand'] == marca_seleccionada]['Model'].unique().tolist()

# Pedir el modelo dentro de los disponibles para la marca seleccionada
modelo_seleccionado = solicitar_input(f"Introduce el modelo del coche ({', '.join(modelos_validos)}): ", str, modelos_validos)

# Seguir pidiendo el resto de datos
nuevo_coche = {
    'Brand': marca_seleccionada,
    'Model': modelo_seleccionado,
    'Year': solicitar_input("Introduce el año del coche: ", int),
    'Engine_Size': solicitar_input("Introduce el tamaño del motor (en litros): ", float),
    'Fuel_Type': solicitar_input(f"Introduce el tipo de combustible ({', '.join(combustibles_validos)}): ", str, combustibles_validos),
    'Transmission': solicitar_input(f"Introduce el tipo de transmisión ({', '.join(transmisiones_validas)}): ", str, transmisiones_validas),
    'Mileage': solicitar_input("Introduce el kilometraje en kilómetros: ", int) * 0.621371, # Conversión a millas (Más cómodo para el usuario).
    'Doors': solicitar_input("Introduce el número de puertas: ", int),
    'Owner_Count': solicitar_input("Introduce el número de dueños anteriores: ", int)
}

# Convertir el coche a DataFrame
nuevo_coche_df = pd.DataFrame([nuevo_coche])

# Convertir las variables categóricas a su codificación numérica
categorical_columns = ['Brand', 'Model', 'Fuel_Type', 'Transmission']
for column in categorical_columns:
    nuevo_coche_df[column] = label_encoders[column].transform(nuevo_coche_df[column])

# Asegurar que las columnas están en el mismo orden que el modelo espera
nuevo_coche_df = nuevo_coche_df[model.feature_names_in_]

# Realizar la predicción
prediccion = model.predict(nuevo_coche_df)

print(f'El precio estimado del coche es: {prediccion[0]:,.2f} $')

Introduce la marca del coche (Audi, BMW, Chevrolet, Ford, Honda, Hyundai, Kia, Mercedes, Toyota, Volkswagen):  Opel


Error: El valor 'Opel' no es válido. Opciones disponibles: Audi, BMW, Chevrolet, Ford, Honda, Hyundai, Kia, Mercedes, Toyota, Volkswagen


Introduce la marca del coche (Audi, BMW, Chevrolet, Ford, Honda, Hyundai, Kia, Mercedes, Toyota, Volkswagen):  Mercedes
Introduce el modelo del coche (GLA, E-Class, C-Class):  GLA
Introduce el año del coche:  2022
Introduce el tamaño del motor (en litros):  3
Introduce el tipo de combustible (Diesel, Electric, Hybrid, Petrol):  Hybrid
Introduce el tipo de transmisión (Automatic, Manual, Semi-Automatic):  Automatic
Introduce el kilometraje en kilómetros:  0
Introduce el número de puertas:  4
Introduce el número de dueños anteriores:  0


El precio estimado del coche es: 15,644.33 $
